In [ ]:
pip install transformers torch accelerate llama-index arxiv pypdf


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.4 MB/s eta 0:00:00
   

In [3]:
import os
import torch
import PyPDF2
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.core import (
    VectorStoreIndex,
    ServiceContext
)
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts import PromptTemplate

def extract_pdf_text(pdf_path):
    """
    Extract text from PDF using PyPDF2

    Args:
        pdf_path (str): Path to the PDF file

    Returns:
        str: Extracted text from the PDF
    """
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # Extract text from all pages
        full_text = []
        for page in reader.pages:
            full_text.append(page.extract_text())

        return "\n".join(full_text)

def summarize_pdf(pdf_path, model_name="facebook/opt-350m"):
    """
    Summarize PDF using open-source LLM

    Args:
        pdf_path (str): Path to the PDF file
        model_name (str): Hugging Face model to use

    Returns:
        dict: Comprehensive and section summaries
    """
    # Extract text from PDF
    pdf_text = extract_pdf_text(pdf_path)

    # Configure device
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map=device,
        torch_dtype=torch.float16 if device == "cuda" else torch.float32
    )

    # Configure LLamaIndex LLM
    llm = HuggingFaceLLM(
        model=model,
        tokenizer=tokenizer,
        context_window=2048,
        max_new_tokens=512,
        generate_kwargs={"temperature": 0.1, "do_sample": False}
    )

    # Comprehensive summary prompt
    comprehensive_prompt = PromptTemplate(
        "Provide a detailed academic-style summary of this research paper. "
        "Include key points about the research question, methodology, "
        "key findings, and potential implications:\n\n{context}\n\n"
        "Comprehensive Summary:"
    )

    # Section summary prompt
    section_prompt = PromptTemplate(
        "Summarize the following section of the research paper. "
        "Focus on the main ideas and critical information:\n\n{context}\n\n"
        "Section Summary:"
    )

    # Generate comprehensive summary
    comprehensive_prompt_filled = comprehensive_prompt.format(context=pdf_text[:4000])
    comprehensive_summary = llm.complete(comprehensive_prompt_filled)

    # Generate section summaries (split text into chunks)
    section_summaries = []
    text_chunks = [pdf_text[i:i+2000] for i in range(0, len(pdf_text), 2000)]
    for chunk in text_chunks[:3]:  # Limit to first 3 chunks
        section_prompt_filled = section_prompt.format(context=chunk)
        section_summary = llm.complete(section_prompt_filled)
        section_summaries.append(str(section_summary))

    return {
        "comprehensive_summary": str(comprehensive_summary),
        "section_summaries": section_summaries
    }

def main():
    pdf_path = "./sample_data/2312.00812.pdf"

    try:
        summaries = summarize_pdf(pdf_path)

        print("🔍 Comprehensive Summary:")
        print(summaries['comprehensive_summary'])

        print("\n📋 Section Summaries:")
        for idx, summary in enumerate(summaries['section_summaries'], 1):
            print(f"Section {idx}:\n{summary}\n{'='*50}\n")

    except Exception as e:
        print(f"Error during PDF summarization: {e}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


🔍 Comprehensive Summary:

The present study investigates the impact of the use of large language models (LLMs) in autonomous driving. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, which is equipped with a large number of LLMs. The study is based on a simulation of a real-world autonomous vehicle (AV) system, whi

In [2]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00
